In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import re
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
data=pd.read_csv('raw_data.csv')
com=pd.read_csv('data.csv')
features=pd.read_csv('features.csv')

In [3]:
def I_label(score):
    if(score==6 or score==7):
        return 0
    if(score>=2 and score <=5):
        return 1
    else:
        return 2

In [4]:
negative_questions=['BXIZZB001V010002','BXIZZB001V010002','BXIZZB001V010010','BXIZZB001V010014','BXIZZB001V010018'
                    ,'BXIZZB001V010020','BXIZZB001V010022','BXIZZB001V010024','BXIZZB001V010026','BXIZZB001V010021',
                    'BXIZZB001V010025','BXIZZB001V010035','BXIZZB001V010040','BXIZZB001V010046','BXIZZB001V010052',
                   'BXIZZB002V010019', 'BXIZZB002V010001','BXIZZB002V010011','BXIZZB002V010014','CXIZZB004V010006',
                    'CXIZZB004V010016','CXIZZB004V010026','CXIZZB004V010036','CXIZZB004V010046','CXIZZB004V010012',
                   'CXIZZB004V010022','CXIZZB004V010032','CXIZZB004V010008','CXIZZB004V010018','CXIZZB004V010028','CXIZZB004V010038',
                   'CXIZZB004V010004','CXIZZB004V010014','CXIZZB004V010024','CXIZZB004V010029','CXIZZB004V010034','CXIZZB004V010039',
                   'CXIZZB004V010044','CXIZZB004V010049','CXIZZB004V010010','CXIZZB004V010020','CXIZZB004V010030']

In [5]:
def C_label(score):
    if(score>=4 and score<=7):
        return 0
    elif(score==2 or score==3):
        return 1
    else:
        return 2

In [6]:
def A_label(score):
    if(score>=4 and score<=6):
        return 0
    if(score>=7 and score <=9):
        return 1
    else:
        return 2

In [7]:
def AM_LC_label(score):
    if(score>=4 and score<=5):
        return 0
    elif(score>=2.6 and score<=3.9):
        return 1
    else:
        return 2

In [8]:
def rest_label(score):
    if(score>=4 and score<=6):
        return 0
    elif(score>=2.6 and score<=3.9):
        return 1
    else:
        return 2

In [9]:
def EN_label(score):
    if(score>=0 and score<=25):
        return 0
    elif(score>=26 and score<=40):
        return 1
    else:
        return 2

In [10]:
def AO_label(score):
    if(score>=0 and score<=30):
        return 0
    elif(score>=31 and score<=40):
        return 1
    else:
        return 2

In [11]:
def C3_label(score):
    if(score>=36 and score<=50):
        return 0
    elif(score>=26 and score<=35):
        return 1
    else:
        return 2

In [12]:
test1_mapping={
    'most people':3,
    'many people':2,
    'some people':1,
    'a few people':0,
    'one or two people':-1,
    'nobody':-2,
    'usually': 3,
    'often':2,
    'sometimes':1,
    'occasionally':0,
    'rarely':-1,
    'never':-2
}

In [13]:
test3_mapping={
    'disagree':-2,
    'slightly disagree':-1,
    'neutral':0,
    'slightly agree':1,
    'agree':2
}

In [14]:
def find_max(A):
    maxn=0
    (row,col)=(-1,-1)
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            if(A[i][j]>=maxn):
                maxn=A[i][j]
                (row,col)=(i,j)
    return (row,col)

In [15]:
def pick_features(matrix,code,threshold):
    row,col=find_max(matrix)
    val=matrix[row][col]
    selected=[]
    while(val>=threshold and row>=0 and col>=0):
        maxn1=0
        for i in range(matrix.shape[0]):
            if(i!=col):
                temp=matrix[row][i]
                if(temp>maxn1):
                    maxn1=temp
        maxn2=0
        for i in range(matrix.shape[0]):
            if(i!=row):
                temp=matrix[col][i]
                if(temp>maxn2):
                    maxn2=temp
        if(maxn1<maxn2):
            selected.append(code[row])
        else:
            selected.append(code[col])
        matrix=np.delete(matrix,[row,col],axis=0)
        matrix=np.delete(matrix,[row,col],axis=1)
        newlist=[]
        for i in range(len(code)):
            if(i!=row and i!=col):
                newlist.append(code[i])
        code=newlist[:]
        row,col=find_max(matrix)
        if(row>=0 and col>=0):
            val=matrix[row][col]
    for c in code:
        selected.append(c)
    return selected

In [16]:
def cluster_distance(i,cluster,distance):
    d=0
    k=len(cluster)
    if(k==0):
        return np.inf
    for c in cluster:
        d=d+distance[i][c]
    return (d/k)

In [17]:
def assign_cluster(i,clusters,distance):
    assign=-1
    minn=np.inf
    j=0
    for cluster in clusters:
        d=cluster_distance(i,cluster,distance)
        if(d<minn):
            minn=d
            assign=j
        j=j+1
    return assign

In [18]:
def cluster_features(n,k,distance):
    labels=np.random.randint(0,k,size=n)
    clusters=[[] for i in range(k)]
    for i in range(n):
        l=labels[i]
        clusters[l].append(i)
    cnt=0
    while(True):
        cnt=cnt+1
        new_labels=np.zeros(n,dtype=int)
        for i in range(n):
            new_labels[i]=assign_cluster(i,clusters,distance)
        new_clusters=[[] for i in range(k)]
        for i in range(n):
            labels[i]=new_labels[i]
            new_clusters[new_labels[i]].append(i)
        to_remove=[]
        for i in range(k):
            if(len(new_clusters[i])==0):
                to_remove.append(i)
        new_clusters = [element for index, element in enumerate(new_clusters) if index not in to_remove]
        k=k-len(to_remove)
        s1=set()
        for i in clusters:
            s1.add(tuple(i))
        s2=set()
        for i in new_clusters:
            s2.add(tuple(i))
        if(s1==s2 or cnt>=100):
            break
        clusters=list(new_clusters)
    return (labels,clusters)

In [19]:
def silhouette(labels,clusters,distance):
    n=len(labels)
    k=len(clusters)
    sill=0
    for i in range(n):
        c=labels[i]
        d=0
        for point in clusters[c]:
            d=d+distance[i][point]
        a=d/len(clusters[c])
        minn=np.inf
        for j in range(k):
            if(c!=j):
                d=cluster_distance(i,clusters[j],distance)
                if(d<minn):
                    minn=d
        b=minn
        s=(b-a)/max(a,b)
        sill=sill+s
    return (sill/n)

In [20]:
def test1_3(trait,codelist,label,feature_mapping):
    Y=[label(x) for x in list(com[trait])]
    df=data[data['Code'].isin(codelist)]
    X=[]
    cnt=0
    to_delete=[]
    for column in df.columns[2:]:
        values=list(df[column])
        l=len(values)
        x=[]
        for i in range(l):
            value=values[i]
            res = re.sub(' +', ' ',value)
            res=res.lower()
            res=''.join(x for x in res if (x.isalpha()or x==' '))
            res=res.strip()
            if(res not in feature_mapping):
                x=[]
                break
            else:
                x.append(feature_mapping[res])
        if(len(x)>0):
            X.append(x)
        else:
            to_delete.append(cnt)
        cnt=cnt+1
    new_y=[]
    for i in range(len(Y)):
        if(i not in to_delete):
            new_y.append(Y[i])
    X_train, X_test, y_train, y_test = train_test_split(X, new_y, test_size=0.1, random_state=42)
    print(len(X))
    svm_classifier = SVC(kernel='rbf', C=1.0, random_state=42)
    svm_classifier.fit(X_train, y_train)
    y_pred = svm_classifier.predict(X_train)
    print("Support Vector Machines: ")
    print("Training accuracy: "+str(accuracy_score(y_train, y_pred)))
    y_pred = svm_classifier.predict(X_test)
    print("Testing accuracy: "+str(accuracy_score(y_test, y_pred)))
    random_forest_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
    random_forest_classifier.fit(X_train, y_train)
    y_pred = random_forest_classifier.predict(X_train)
    print("Random Forest Classifier :")
    print("Training accuracy: "+str(accuracy_score(y_train, y_pred)))
    y_pred = random_forest_classifier.predict(X_test)
    print("Testing accuracy: "+str(accuracy_score(y_test, y_pred)))
    logistic_regression_classifier = LogisticRegression(max_iter=1000, random_state=42)
    logistic_regression_classifier.fit(X_train, y_train)
    y_pred = logistic_regression_classifier.predict(X_train)
    print("Logistic Regression Classifier :")
    print("Training accuracy: "+str(accuracy_score(y_train, y_pred)))
    y_pred = logistic_regression_classifier.predict(X_test)
    print("Testing accuracy: "+str(accuracy_score(y_test, y_pred)))
    mlp = MLPClassifier(hidden_layer_sizes=(100,50), max_iter=500, random_state=42)
    mlp.fit(X_train, y_train)
    y_pred = mlp.predict(X_train)
    print("Multi Layer Perceptron :")
    print("Training accuracy: "+str(accuracy_score(y_train, y_pred)))
    y_pred = mlp.predict(X_test)
    print("Testing accuracy: "+str(accuracy_score(y_test, y_pred)))

In [21]:
def reduced_test13(trait,start,num_features,label,feature_mapping):
    codelist=[]
    y=[]
    for i in range(start,start+num_features):
        codelist.append(features['Code'][i])
    df=data[data['Code'].isin(codelist)]
    to_delete=[]
    cnt=0
    for i in df.index:
        x=[]
        for column in df.columns[2:]:
            val=df[column][i]
            if not pd.isna(val):
                res = re.sub(' +', ' ',val)
                res=res.lower()
                res=''.join(c for c in res if (c.isalpha()or c==' '))
                res=res.strip()
                if(res in feature_mapping):
#                     if(df['Code'][i] in negative_questions):
#                         x.append(-1*feature_mapping[res])
#                     else:
                    x.append(feature_mapping[res])
                else:
                    x.append(-1)
                    to_delete.append(cnt)
                    cnt=cnt+1
        y.append(x)
    new_y=[]
    for x in y:
        x_new=[]
        for i in range(len(x)):
            if(i not in to_delete):
                x_new.append(x[i])
        new_y.append(x_new)
    y=np.array(new_y)
    corr_matrix=[]
    for i in range(y.shape[0]):
        row=[]
        x1=y[i]
        for j in range(y.shape[0]):
            if(i==j):
                row.append(0)
                continue
            x2=y[j]
            row.append(abs(np.corrcoef(x1,x2)[0][1]))
        corr_matrix.append(row)
    corr_matrix=np.array(corr_matrix)
    featureslist=pick_features(corr_matrix,codelist,0.1)
    print(featureslist)
    test1_3(trait,featureslist,label,feature_mapping)
    dist_matrix=[]
    for i in range(y.shape[0]):
        row=[]
        x1=y[i]
        for j in range(y.shape[0]):
            x2=y[j]
            row.append(abs(np.corrcoef(x1,x2)[0][1]))
        dist_matrix.append(row)
    dist_matrix=np.array(dist_matrix)
    maxn=-1
    k=1
    for i in range(2,10):
        labels,clusters=cluster_features(num_features,i,dist_matrix)
        sill=silhouette(labels,clusters,dist_matrix)
        if(sill>maxn):
            maxn=sill
            k=i
    labels,clusters=cluster_features(num_features,k,dist_matrix)
    for cluster in clusters:
        newlist=[]
        if(len(cluster)==0):
            continue
        for c in cluster:
            newlist.append(codelist[c])
        print(newlist)
        test1_3(trait,newlist,label,feature_mapping)

In [22]:
reduced_test13('EI',0,9,I_label,test1_mapping)

['BXIZZB001V010013', 'BXIZZB001V010005', 'BXIZZB001V010001', 'BXIZZB001V010009', 'BXIZZB001V010015']
4546
Support Vector Machines: 
Training accuracy: 0.786115864091909
Testing accuracy: 0.7758241758241758
Random Forest Classifier :
Training accuracy: 0.8704473233928135
Testing accuracy: 0.734065934065934
Logistic Regression Classifier :
Training accuracy: 0.7301393302371059
Testing accuracy: 0.7230769230769231
Multi Layer Perceptron :
Training accuracy: 0.8078709362014177
Testing accuracy: 0.778021978021978
['BXIZZB001V010015']
4546
Support Vector Machines: 
Training accuracy: 0.5534099242239061
Testing accuracy: 0.5406593406593406
Random Forest Classifier :
Training accuracy: 0.5534099242239061
Testing accuracy: 0.5406593406593406
Logistic Regression Classifier :
Training accuracy: 0.54705450989978
Testing accuracy: 0.5538461538461539
Multi Layer Perceptron :
Training accuracy: 0.5534099242239061
Testing accuracy: 0.5406593406593406
['BXIZZB001V010001', 'BXIZZB001V010003', 'BXIZZB001

In [23]:
reduced_test13('WI',9,9,I_label,test1_mapping)

['BXIZZB001V010048', 'BXIZZB001V010034', 'BXIZZB001V010028', 'BXIZZB001V010037', 'BXIZZB001V010042']
4329
Support Vector Machines: 
Training accuracy: 0.7831108829568788
Testing accuracy: 0.7782909930715936
Random Forest Classifier :
Training accuracy: 0.8662731006160165
Testing accuracy: 0.7644341801385681
Logistic Regression Classifier :
Training accuracy: 0.571611909650924
Testing accuracy: 0.5219399538106235
Multi Layer Perceptron :
Training accuracy: 0.8157084188911704
Testing accuracy: 0.7967667436489607
['BXIZZB001V010034', 'BXIZZB001V010037', 'BXIZZB001V010039', 'BXIZZB001V010045', 'BXIZZB001V010048', 'BXIZZB001V010051']
4315
Support Vector Machines: 
Training accuracy: 0.8194694823590007
Testing accuracy: 0.8055555555555556
Random Forest Classifier :
Training accuracy: 0.9235127478753541
Testing accuracy: 0.8032407407407407
Logistic Regression Classifier :
Training accuracy: 0.47952613958279683
Testing accuracy: 0.5300925925925926
Multi Layer Perceptron :
Training accuracy: 0.

In [24]:
reduced_test13('EC',18,9,C_label,test1_mapping)

['BXIZZB001V010054', 'BXIZZB001V010030', 'BXIZZB001V010036', 'BXIZZB001V010044', 'BXIZZB001V010033']
4257
Support Vector Machines: 
Training accuracy: 0.8125815713912816
Testing accuracy: 0.8075117370892019
Random Forest Classifier :
Training accuracy: 0.9083790133124511
Testing accuracy: 0.8075117370892019
Logistic Regression Classifier :
Training accuracy: 0.7290524667188724
Testing accuracy: 0.7394366197183099
Multi Layer Perceptron :
Training accuracy: 0.855912294440094
Testing accuracy: 0.7981220657276995
['BXIZZB001V010033']
4416
Support Vector Machines: 
Training accuracy: 0.5908404630095622
Testing accuracy: 0.6402714932126696
Random Forest Classifier :
Training accuracy: 0.5908404630095622
Testing accuracy: 0.6402714932126696
Logistic Regression Classifier :
Training accuracy: 0.5888273779567187
Testing accuracy: 0.6380090497737556
Multi Layer Perceptron :
Training accuracy: 0.5908404630095622
Testing accuracy: 0.6402714932126696
['BXIZZB001V010030', 'BXIZZB001V010036', 'BXIZZ

In [25]:
reduced_test13('WC',27,9,C_label,test1_mapping)

['BXIZZB001V010006', 'BXIZZB001V010010', 'BXIZZB001V010018', 'BXIZZB001V010014', 'BXIZZB001V010002']
4384
Support Vector Machines: 
Training accuracy: 0.7731305449936628
Testing accuracy: 0.765375854214123
Random Forest Classifier :
Training accuracy: 0.9082382762991128
Testing accuracy: 0.7585421412300684
Logistic Regression Classifier :
Training accuracy: 0.6610899873257288
Testing accuracy: 0.6674259681093394
Multi Layer Perceptron :
Training accuracy: 0.84106463878327
Testing accuracy: 0.785876993166287
['BXIZZB001V010014', 'BXIZZB001V010024', 'BXIZZB001V010026']
4274
Support Vector Machines: 
Training accuracy: 0.702808112324493
Testing accuracy: 0.7336448598130841
Random Forest Classifier :
Training accuracy: 0.7210088403536141
Testing accuracy: 0.705607476635514
Logistic Regression Classifier :
Training accuracy: 0.6383255330213209
Testing accuracy: 0.6822429906542056
Multi Layer Perceptron :
Training accuracy: 0.7139885595423817
Testing accuracy: 0.7266355140186916
['BXIZZB001V

In [26]:
reduced_test13('EA',36,9,A_label,test1_mapping)

['BXIZZB001V010021', 'BXIZZB001V010012', 'BXIZZB001V010008', 'BXIZZB001V010025', 'BXIZZB001V010017']
4284
Support Vector Machines: 
Training accuracy: 0.8261997405966277
Testing accuracy: 0.7738927738927739
Random Forest Classifier :
Training accuracy: 0.9009079118028535
Testing accuracy: 0.7808857808857809
Logistic Regression Classifier :
Training accuracy: 0.7424124513618677
Testing accuracy: 0.6713286713286714
Multi Layer Perceptron :
Training accuracy: 0.8664072632944229
Testing accuracy: 0.8135198135198135
['BXIZZB001V010019', 'BXIZZB001V010025']
4283
Support Vector Machines: 
Training accuracy: 0.5212765957446809
Testing accuracy: 0.49417249417249415
Random Forest Classifier :
Training accuracy: 0.5303580695381422
Testing accuracy: 0.4965034965034965
Logistic Regression Classifier :
Training accuracy: 0.3679294239750908
Testing accuracy: 0.3939393939393939
Multi Layer Perceptron :
Training accuracy: 0.52854177477945
Testing accuracy: 0.5058275058275058
['BXIZZB001V010004', 'BXIZZ

In [27]:
reduced_test13('WA',45,9,A_label,test1_mapping)

['BXIZZB001V010038', 'BXIZZB001V010052', 'BXIZZB001V010043', 'BXIZZB001V010035', 'BXIZZB001V010029']
4262
Support Vector Machines: 
Training accuracy: 0.8294654498044328
Testing accuracy: 0.8079625292740047
Random Forest Classifier :
Training accuracy: 0.913168187744459
Testing accuracy: 0.8173302107728337
Logistic Regression Classifier :
Training accuracy: 0.7692307692307693
Testing accuracy: 0.7704918032786885
Multi Layer Perceptron :
Training accuracy: 0.8737940026075619
Testing accuracy: 0.8079625292740047
['BXIZZB001V010032', 'BXIZZB001V010043', 'BXIZZB001V010049', 'BXIZZB001V010052']
4455
Support Vector Machines: 
Training accuracy: 0.7922175106011474
Testing accuracy: 0.7713004484304933
Random Forest Classifier :
Training accuracy: 0.8341232227488151
Testing accuracy: 0.7466367713004485
Logistic Regression Classifier :
Training accuracy: 0.7485657271139935
Testing accuracy: 0.7331838565022422
Multi Layer Perceptron :
Training accuracy: 0.8054377650286855
Testing accuracy: 0.7713

In [28]:
reduced_test13('E',74,10,EN_label,test3_mapping)

['CXIZZB004V010006', 'CXIZZB004V010001', 'CXIZZB004V010031', 'CXIZZB004V010041', 'CXIZZB004V010016']
4546
Support Vector Machines: 
Training accuracy: 0.8919579564898558
Testing accuracy: 0.8857142857142857
Random Forest Classifier :
Training accuracy: 0.9269127352725495
Testing accuracy: 0.8835164835164835
Logistic Regression Classifier :
Training accuracy: 0.8841359080909313
Testing accuracy: 0.8681318681318682
Multi Layer Perceptron :
Training accuracy: 0.9007577609386458
Testing accuracy: 0.8813186813186813
['CXIZZB004V010006', 'CXIZZB004V010016', 'CXIZZB004V010026', 'CXIZZB004V010036', 'CXIZZB004V010041', 'CXIZZB004V010046']
4545
Support Vector Machines: 
Training accuracy: 0.8948655256723717
Testing accuracy: 0.843956043956044
Random Forest Classifier :
Training accuracy: 0.9767726161369193
Testing accuracy: 0.8725274725274725
Logistic Regression Classifier :
Training accuracy: 0.8713936430317848
Testing accuracy: 0.8373626373626374
Multi Layer Perceptron :
Training accuracy: 0.9

In [29]:
reduced_test13('A',84,10,AO_label,test3_mapping)

['CXIZZB004V010037', 'CXIZZB004V010042', 'CXIZZB004V010012', 'CXIZZB004V010002', 'CXIZZB004V010027']
4546
Support Vector Machines: 
Training accuracy: 0.8790026888291371
Testing accuracy: 0.8879120879120879
Random Forest Classifier :
Training accuracy: 0.9044243461256416
Testing accuracy: 0.8681318681318682
Logistic Regression Classifier :
Training accuracy: 0.8658029821559521
Testing accuracy: 0.8835164835164835
Multi Layer Perceptron :
Training accuracy: 0.8902468834025911
Testing accuracy: 0.8791208791208791
['CXIZZB004V010002', 'CXIZZB004V010007', 'CXIZZB004V010042', 'CXIZZB004V010047']
4546
Support Vector Machines: 
Training accuracy: 0.8501588853581031
Testing accuracy: 0.8527472527472527
Random Forest Classifier :
Training accuracy: 0.8579809337570277
Testing accuracy: 0.8527472527472527
Logistic Regression Classifier :
Training accuracy: 0.8533365925201662
Testing accuracy: 0.8527472527472527
Multi Layer Perceptron :
Training accuracy: 0.852603275482767
Testing accuracy: 0.8571

In [30]:
reduced_test13('C',94,10,C3_label,test3_mapping)

['CXIZZB004V010008', 'CXIZZB004V010013', 'CXIZZB004V010048', 'CXIZZB004V010028', 'CXIZZB004V010003']
4545
Support Vector Machines: 
Training accuracy: 0.8102689486552567
Testing accuracy: 0.8
Random Forest Classifier :
Training accuracy: 0.8594132029339854
Testing accuracy: 0.7802197802197802
Logistic Regression Classifier :
Training accuracy: 0.8009779951100244
Testing accuracy: 0.7692307692307693
Multi Layer Perceptron :
Training accuracy: 0.830562347188264
Testing accuracy: 0.7912087912087912
['CXIZZB004V010003', 'CXIZZB004V010013', 'CXIZZB004V010018', 'CXIZZB004V010023', 'CXIZZB004V010028', 'CXIZZB004V010048']
4546
Support Vector Machines: 
Training accuracy: 0.836225861647519
Testing accuracy: 0.8219780219780219
Random Forest Classifier :
Training accuracy: 0.9022243950134441
Testing accuracy: 0.8131868131868132
Logistic Regression Classifier :
Training accuracy: 0.8222928379369348
Testing accuracy: 0.8065934065934066
Multi Layer Perceptron :
Training accuracy: 0.8662918601808849


In [31]:
reduced_test13('N',104,10,EN_label,test3_mapping)

['CXIZZB004V010049', 'CXIZZB004V010034', 'CXIZZB004V010024', 'CXIZZB004V010014', 'CXIZZB004V010019']
4546
Support Vector Machines: 
Training accuracy: 0.8895135663651919
Testing accuracy: 0.8791208791208791
Random Forest Classifier :
Training accuracy: 0.9371791737961379
Testing accuracy: 0.865934065934066
Logistic Regression Classifier :
Training accuracy: 0.8765582987044732
Testing accuracy: 0.8747252747252747
Multi Layer Perceptron :
Training accuracy: 0.9122463945245661
Testing accuracy: 0.8571428571428571
['CXIZZB004V010029']
4546
Support Vector Machines: 
Training accuracy: 0.7680273771693963
Testing accuracy: 0.7670329670329671
Random Forest Classifier :
Training accuracy: 0.7680273771693963
Testing accuracy: 0.7670329670329671
Logistic Regression Classifier :
Training accuracy: 0.7680273771693963
Testing accuracy: 0.7670329670329671
Multi Layer Perceptron :
Training accuracy: 0.7680273771693963
Testing accuracy: 0.7670329670329671
['CXIZZB004V010004', 'CXIZZB004V010014', 'CXIZZ

In [32]:
reduced_test13('O',114,10,AO_label,test3_mapping)

['CXIZZB004V010050', 'CXIZZB004V010015', 'CXIZZB004V010040', 'CXIZZB004V010010', 'CXIZZB004V010005']
4546
Support Vector Machines: 
Training accuracy: 0.9156685406990955
Testing accuracy: 0.9010989010989011
Random Forest Classifier :
Training accuracy: 0.9320459545343437
Testing accuracy: 0.9054945054945055
Logistic Regression Classifier :
Training accuracy: 0.9161574187240283
Testing accuracy: 0.9076923076923077
Multi Layer Perceptron :
Training accuracy: 0.9217795160107554
Testing accuracy: 0.9186813186813186
['CXIZZB004V010010', 'CXIZZB004V010020', 'CXIZZB004V010030', 'CXIZZB004V010040']
4546
Support Vector Machines: 
Training accuracy: 0.8904913224150575
Testing accuracy: 0.8681318681318682
Random Forest Classifier :
Training accuracy: 0.9041799071131753
Testing accuracy: 0.8681318681318682
Logistic Regression Classifier :
Training accuracy: 0.8895135663651919
Testing accuracy: 0.865934065934066
Multi Layer Perceptron :
Training accuracy: 0.8941579076020533
Testing accuracy: 0.8637

In [33]:
def test2(trait,codelist,label):
    Y=[label(x) for x in list(com[trait])]
    df=data[data['Code'].isin(codelist)]
    X=[]
    for column in df.columns[2:]:
        values=list(df[column])
        l=len(values)
        x=[]
        for i in range(l):
            value=int(values[i])
            x.append(value)
        X.append(x)
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42)
    print(len(X))
    svm_classifier = SVC(kernel='rbf', C=1.0, random_state=42)
    svm_classifier.fit(X_train, y_train)
    y_pred = svm_classifier.predict(X_train)
    print("Support Vector Machines: ")
    print("Training accuracy: "+str(accuracy_score(y_train, y_pred)))
    y_pred = svm_classifier.predict(X_test)
    print("Testing accuracy: "+str(accuracy_score(y_test, y_pred)))
    random_forest_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
    random_forest_classifier.fit(X_train, y_train)
    y_pred = random_forest_classifier.predict(X_train)
    print("Random Forest Classifier :")
    print("Training accuracy: "+str(accuracy_score(y_train, y_pred)))
    y_pred = random_forest_classifier.predict(X_test)
    print("Testing accuracy: "+str(accuracy_score(y_test, y_pred)))
    logistic_regression_classifier = LogisticRegression(max_iter=1000, random_state=42)
    logistic_regression_classifier.fit(X_train, y_train)
    y_pred = logistic_regression_classifier.predict(X_train)
    print("Logistic Regression Classifier :")
    print("Training accuracy: "+str(accuracy_score(y_train, y_pred)))
    y_pred = logistic_regression_classifier.predict(X_test)
    print("Testing accuracy: "+str(accuracy_score(y_test, y_pred)))
    mlp = MLPClassifier(hidden_layer_sizes=(100,50), max_iter=500, random_state=42)
    mlp.fit(X_train, y_train)
    y_pred = mlp.predict(X_train)
    print("Multi Layer Perceptron :")
    print("Training accuracy: "+str(accuracy_score(y_train, y_pred)))
    y_pred = mlp.predict(X_test)
    print("Testing accuracy: "+str(accuracy_score(y_test, y_pred)))

In [34]:
def reduced_test2(trait,start,num_features,label):
    codelist=[]
    y=[]
    for i in range(start,start+num_features):
        codelist.append(features['Code'][i])
    df=data[data['Code'].isin(codelist)]
    for i in df.index:
        x=[]
        for column in df.columns[2:]:
            val=df[column][i]
#             if(df['Code'][i] in negative_questions):
#                 x.append(-1*int(val))
#             else:
            x.append(int(val))
        y.append(x)
    y=np.array(y)
    corr_matrix=[]
    for i in range(y.shape[0]):
        row=[]
        x1=y[i]
        for j in range(y.shape[0]):
            if(i==j):
                row.append(0)
                continue
            x2=y[j]
            row.append(abs(np.corrcoef(x1,x2)[0][1]))
        corr_matrix.append(row)
    corr_matrix=np.array(corr_matrix)
    featureslist=pick_features(corr_matrix,codelist,0.1)
    print(featureslist)
    test2(trait,featureslist,label)
    dist_matrix=[]
    for i in range(y.shape[0]):
        row=[]
        x1=y[i]
        for j in range(y.shape[0]):
            x2=y[j]
            row.append(abs(np.corrcoef(x1,x2)[0][1]))
        dist_matrix.append(row)
    dist_matrix=np.array(dist_matrix)
    maxn=-1
    k=1
    for i in range(2,10):
        labels,clusters=cluster_features(num_features,i,dist_matrix)
        sill=silhouette(labels,clusters,dist_matrix)
        if(sill>maxn):
            maxn=sill
            k=i
    labels,clusters=cluster_features(num_features,k,dist_matrix)
    for cluster in clusters:
        newlist=[]
        if(len(cluster)==0):
            continue
        for c in cluster:
            newlist.append(codelist[c])
        print(newlist)
        test2(trait,newlist,label)

In [35]:
reduced_test2('AM',54,3,AM_LC_label)

['BXIZZB002V010003', 'BXIZZB002V010019']
4546
Support Vector Machines: 
Training accuracy: 0.9029577120508433
Testing accuracy: 0.8857142857142857
Random Forest Classifier :
Training accuracy: 0.9063798582253728
Testing accuracy: 0.8835164835164835
Logistic Regression Classifier :
Training accuracy: 0.8848692251283304
Testing accuracy: 0.8681318681318682
Multi Layer Perceptron :
Training accuracy: 0.9044243461256416
Testing accuracy: 0.8835164835164835
['BXIZZB002V010003', 'BXIZZB002V010007']
4546
Support Vector Machines: 
Training accuracy: 0.75287215839648
Testing accuracy: 0.7384615384615385
Random Forest Classifier :
Training accuracy: 0.75287215839648
Testing accuracy: 0.7384615384615385
Logistic Regression Classifier :
Training accuracy: 0.75287215839648
Testing accuracy: 0.7384615384615385
Multi Layer Perceptron :
Training accuracy: 0.75287215839648
Testing accuracy: 0.7384615384615385
['BXIZZB002V010019']
4546
Support Vector Machines: 
Training accuracy: 0.8506477633830359
Test

In [36]:
reduced_test2('LC',57,4,AM_LC_label)

['BXIZZB002V010011', 'BXIZZB002V010006']
4546
Support Vector Machines: 
Training accuracy: 0.9325348325592765
Testing accuracy: 0.9362637362637363
Random Forest Classifier :
Training accuracy: 0.9325348325592765
Testing accuracy: 0.9362637362637363
Logistic Regression Classifier :
Training accuracy: 0.9325348325592765
Testing accuracy: 0.9362637362637363
Multi Layer Perceptron :
Training accuracy: 0.9325348325592765
Testing accuracy: 0.9362637362637363
['BXIZZB002V010001', 'BXIZZB002V010011', 'BXIZZB002V010014']
4546
Support Vector Machines: 
Training accuracy: 0.967978489366903
Testing accuracy: 0.978021978021978
Random Forest Classifier :
Training accuracy: 0.9792226839403568
Testing accuracy: 0.9582417582417583
Logistic Regression Classifier :
Training accuracy: 0.9638230261549743
Testing accuracy: 0.9692307692307692
Multi Layer Perceptron :
Training accuracy: 0.9704228794915668
Testing accuracy: 0.967032967032967
['BXIZZB002V010006']
4546
Support Vector Machines: 
Training accuracy

In [37]:
reduced_test2('MC',61,4,rest_label)

['BXIZZB002V010009', 'BXIZZB002V010002']
4546
Support Vector Machines: 
Training accuracy: 0.7936934734783672
Testing accuracy: 0.7846153846153846
Random Forest Classifier :
Training accuracy: 0.7963823026154975
Testing accuracy: 0.7978021978021979
Logistic Regression Classifier :
Training accuracy: 0.7824492789049132
Testing accuracy: 0.8
Multi Layer Perceptron :
Training accuracy: 0.7939379124908336
Testing accuracy: 0.778021978021978
['BXIZZB002V010002']
4546
Support Vector Machines: 
Training accuracy: 0.6692740161329748
Testing accuracy: 0.6681318681318681
Random Forest Classifier :
Training accuracy: 0.6692740161329748
Testing accuracy: 0.6681318681318681
Logistic Regression Classifier :
Training accuracy: 0.6477633830359325
Testing accuracy: 0.6351648351648351
Multi Layer Perceptron :
Training accuracy: 0.6692740161329748
Testing accuracy: 0.6681318681318681
['BXIZZB002V010009', 'BXIZZB002V010012', 'BXIZZB002V010016']
4546
Support Vector Machines: 
Training accuracy: 0.879247127

In [38]:
reduced_test2('ND',65,2,rest_label)

['BXIZZB002V010010']
4546
Support Vector Machines: 
Training accuracy: 0.7560498655585431
Testing accuracy: 0.7582417582417582
Random Forest Classifier :
Training accuracy: 0.7562943045710095
Testing accuracy: 0.7582417582417582
Logistic Regression Classifier :
Training accuracy: 0.7347836714739673
Testing accuracy: 0.7252747252747253
Multi Layer Perceptron :
Training accuracy: 0.7562943045710095
Testing accuracy: 0.7582417582417582
['BXIZZB002V010010']


C:\Users\JBSCHOLOR-2020-18\AppData\Local\Temp\ipykernel_8108\3520371116.py:18: RuntimeWarning: invalid value encountered in double_scalars
  s=(b-a)/max(a,b)


4546
Support Vector Machines: 
Training accuracy: 0.7560498655585431
Testing accuracy: 0.7582417582417582
Random Forest Classifier :
Training accuracy: 0.7562943045710095
Testing accuracy: 0.7582417582417582
Logistic Regression Classifier :
Training accuracy: 0.7347836714739673
Testing accuracy: 0.7252747252747253
Multi Layer Perceptron :
Training accuracy: 0.7562943045710095
Testing accuracy: 0.7582417582417582
['BXIZZB002V010013']
4546
Support Vector Machines: 
Training accuracy: 0.7809826448301149
Testing accuracy: 0.8
Random Forest Classifier :
Training accuracy: 0.7809826448301149
Testing accuracy: 0.8
Logistic Regression Classifier :
Training accuracy: 0.7523832803715473
Testing accuracy: 0.7648351648351648
Multi Layer Perceptron :
Training accuracy: 0.7809826448301149
Testing accuracy: 0.8


In [39]:
reduced_test2('PfW',67,3,rest_label)

['BXIZZB002V010005', 'BXIZZB002V010015']
4546
Support Vector Machines: 
Training accuracy: 0.8574920557320949
Testing accuracy: 0.8285714285714286
Random Forest Classifier :
Training accuracy: 0.8592031288193596
Testing accuracy: 0.8241758241758241
Logistic Regression Classifier :
Training accuracy: 0.8574920557320949
Testing accuracy: 0.832967032967033
Multi Layer Perceptron :
Training accuracy: 0.8545587875824981
Testing accuracy: 0.832967032967033
['BXIZZB002V010015', 'BXIZZB002V010018']
4546
Support Vector Machines: 
Training accuracy: 0.8601808848692252
Testing accuracy: 0.8483516483516483
Random Forest Classifier :
Training accuracy: 0.8631141530188218
Testing accuracy: 0.8395604395604396
Logistic Regression Classifier :
Training accuracy: 0.8618919579564899
Testing accuracy: 0.8417582417582418
Multi Layer Perceptron :
Training accuracy: 0.8601808848692252
Testing accuracy: 0.8483516483516483
['BXIZZB002V010005']
4546
Support Vector Machines: 
Training accuracy: 0.710584209239794

In [40]:
reduced_test2('SE',70,4,rest_label)

['BXIZZB002V010020', 'BXIZZB002V010004']
4546
Support Vector Machines: 
Training accuracy: 0.7692495722317282
Testing accuracy: 0.7604395604395604
Random Forest Classifier :
Training accuracy: 0.7768271816181863
Testing accuracy: 0.7648351648351648
Logistic Regression Classifier :
Training accuracy: 0.7748716695184551
Testing accuracy: 0.7626373626373626
Multi Layer Perceptron :
Training accuracy: 0.7738939134685896
Testing accuracy: 0.7692307692307693
['BXIZZB002V010004', 'BXIZZB002V010008']
4546
Support Vector Machines: 
Training accuracy: 0.7888046932290393
Testing accuracy: 0.7802197802197802
Random Forest Classifier :
Training accuracy: 0.7910046443412369
Testing accuracy: 0.8043956043956044
Logistic Regression Classifier :
Training accuracy: 0.7866047421168418
Testing accuracy: 0.7846153846153846
Multi Layer Perceptron :
Training accuracy: 0.7900268882913714
Testing accuracy: 0.8
['BXIZZB002V010017', 'BXIZZB002V010020']
4546
Support Vector Machines: 
Training accuracy: 0.77193840